# Benchmark pure pipeline:

Runtime comparison, of the translated pipelines.

## Required packages:
See: requirements/requirements.txt and requirements/requirements.dev.txt

## Some parameters you might want to set:

In [1]:
import pathlib
import timeit
from inspect import cleandoc
import matplotlib.pyplot as plt
from tqdm import tqdm

from mlinspect.to_sql.dbms_connectors.postgresql_connector import PostgresqlConnector
from mlinspect.to_sql.dbms_connectors.umbra_connector import UmbraConnector
from pandas_connector import PandasConnector
from mlinspect import PipelineInspector
from mlinspect.utils import get_project_root
from _code_as_string import get_healthcare_pipe_code
from pandas_connector import PandasConnector
from _benchmark_utility import plot_compare, ROOT_DIR, PLOT_DIR
from _code_as_string import Join, GroupBy, Selection, Projection

DO_CLEANUP = True
SIZES = [(10 ** i) for i in range(2, 8, 1)]
BENCH_REP = 3
MLINSPECT_ROOT_DIR = get_project_root()

# DBMS related:
UMBRA_DIR = r"/home/luca/Documents/Bachelorarbeit/umbra-students"
UMBRA_USER = "postgres"
UMBRA_PW = " "
UMBRA_DB = ""
UMBRA_PORT = 5433
UMBRA_HOST = "/tmp/"

POSTGRES_USER = "luca"
POSTGRES_PW = "password"
POSTGRES_DB = "healthcare_benchmark"
POSTGRES_PORT = 5432
POSTGRES_HOST = "localhost"

# Data Generation

To be able to benchmark and compare the different approaches, some datasets
will need to be generated before. The datasets are just and expansion of the
original ones.

In [2]:
!{sys.executable} -m pip install faker

from data_generation.compas_data_generation import generate_compas_dataset
from data_generation.healthcare_data_generation import generate_healthcare_dataset

# We only generate the files, that are not already existing:

COMPAS_DATA_PATHS = generate_compas_dataset(SIZES)
HEALTHCARE_DATA_PATHS = generate_healthcare_dataset(SIZES)

/bin/bash: {sys.executable}: command not found
Data generated or found for: size = 100 -- compas
Data generated or found for: size = 1000 -- compas
Data generated or found for: size = 10000 -- compas
Data generated or found for: size = 100000 -- compas
Data generated or found for: size = 1000000 -- compas
Data generated or found for: size = 10000000 -- compas
Data generated or found for: size = 100 -- healthcare
Data generated or found for: size = 1000 -- healthcare
Data generated or found for: size = 10000 -- healthcare
Data generated or found for: size = 100000 -- healthcare
Data generated or found for: size = 1000000 -- healthcare
Data generated or found for: size = 10000000 -- healthcare


## Demo:

In [3]:
def get_healthcare_sql_str(pipeline_code, mode, materialize):
    PipelineInspector \
        .on_pipeline_from_string(pipeline_code) \
        .execute_in_sql(dbms_connector=None, mode=mode, materialize=materialize)

    setup_file = \
        pathlib.Path(MLINSPECT_ROOT_DIR / r"mlinspect/to_sql/generated_code/create_table.sql")
    test_file = \
        pathlib.Path(MLINSPECT_ROOT_DIR / r"mlinspect/to_sql/generated_code/pipeline.sql")

    with setup_file.open("r") as file:
        setup_code = file.read()

    with test_file.open("r") as file:
        test_code = file.read()

    return setup_code, test_code

In [ ]:
def pure_pipeline_benchmark(add_impute_and_onehot=False, title = "HealthcarePurePipeComparison"):
    umbra_times = []
    postgres_times = []
    pandas_times = []

    postgres = PostgresqlConnector(dbname="healthcare_benchmark", user="luca", password="password", port=5432,
                                   host="localhost")
    pandas = PandasConnector()

    for i, (path_to_csv_his, path_to_csv_pat) in enumerate(HEALTHCARE_DATA_PATHS):
        print(f"ITERATION: {i} - for table size of: {SIZES[i]}")

        setup_code_orig, test_code_orig = get_healthcare_pipe_code(path_to_csv_his, path_to_csv_pat, add_impute_and_onehot=add_impute_and_onehot)

        setup_code, test_code = get_healthcare_sql_str(setup_code_orig + "\n" + test_code_orig, mode="CTE",
                                                       materialize=False)

        ################################################################################################################
        # time Umbra:
        umbra = UmbraConnector(dbname="", user="postgres", password=" ", port=5433, host="/tmp/", umbra_dir=UMBRA_DIR)
        umbra.run(setup_code)
        umbra_times.append(umbra.benchmark_run(test_code, repetitions=BENCH_REP))

        ################################################################################################################
        # time Postgres:
        postgres.run(setup_code)
        postgres_times.append(postgres.benchmark_run(test_code, repetitions=BENCH_REP))

        ################################################################################################################
        # time Pandas:
        pandas_times.append(pandas.benchmark_run(pandas_code=test_code_orig, setup_code=setup_code_orig,
                                                 repetitions=BENCH_REP))
        ################################################################################################################

    print(f"Plotting..")
    names = ["Umbra", "Postgresql", "Pandas"]
    table = [umbra_times, postgres_times, pandas_times]
    plot_compare(title, SIZES, all_y=table, all_y_names=names, save=True)

pure_pipeline_benchmark()

ITERATION: 0 - for table size of: 100

Just translation to SQL is performed! 
-> SQL-Code placed at: mlinspect/to_sql/generated_code



2021-07-29 08:06:49.244675: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-07-29 08:06:49.244701: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


Executing Query in Umbra...
Done in 0.0006233333333333333!
Executing Query in Postgres...
Done in 2.0386666666666664!
Executing Query in Pandas...
Done in 11.654675668978598!
ITERATION: 1 - for table size of: 1000

Just translation to SQL is performed! 
-> SQL-Code placed at: mlinspect/to_sql/generated_code

Executing Query in Umbra...
Done in 0.0011616666666666668!
Executing Query in Postgres...
Done in 15.445666666666668!
Executing Query in Pandas...
Done in 12.353357332661593!
ITERATION: 2 - for table size of: 10000

Just translation to SQL is performed! 
-> SQL-Code placed at: mlinspect/to_sql/generated_code

Executing Query in Umbra...
Done in 0.0038743333333333334!
Executing Query in Postgres...
Done in 148.79166666666666!
Executing Query in Pandas...
Done in 25.925816664918482!
ITERATION: 3 - for table size of: 100000

Just translation to SQL is performed! 
-> SQL-Code placed at: mlinspect/to_sql/generated_code

Executing Query in Umbra...
Done in 0.013052333333333332!
Executing

### The original results:
**Pipeline runtimes**:

![p_r](plots/HealthcarePurePipeComparison.png)

<img src="plots/HealthcarePurePipeComparison.png" width="60" height="60" />

In [ ]:
pure_pipeline_benchmark(add_impute_and_onehot=True, title="HealthcarePurePipeComparisonSimpImpOneHot")

## When also considering "Simple Imputer" and "OneHotEncoder" from scikit-learn:


### The original results:
**Pipeline runtimes**:

![p_r](plots/HealthcarePurePipeComparisonSimpImpOneHot.png)


In [ ]:
# Clean_up:
if DO_CLEANUP:
    [f.unlink() for f in PLOT_DIR.glob("*_.png") if f.is_file()]